In [1]:
import psycopg2
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from io import StringIO
from surprise import Reader
from surprise import Dataset
from surprise import SVD, SlopeOne, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate, GridSearchCV

pd.options.display.float_format = '{:20,.2f}'.format

MONTHS_AGO = 3

## get df

In [2]:
date_ago = datetime.now() - relativedelta(months=MONTHS_AGO)
date_ago = datetime.strftime(date_ago, format='%Y%m%d')

print(date_ago)

20191126


In [3]:
conn = psycopg2.connect(
    host="*****",
    user="*****",
    dbname="*****",
    password="*****",
    sslmode="require"
)
cur = conn.cursor()

sql_query = "COPY (SELECT * FROM sales WHERE dt > '{}') TO STDOUT WITH (FORMAT CSV, DELIMITER '|', HEADER TRUE)".format(date_ago)
io_object = StringIO()
cur.copy_expert(sql_query, io_object)
io_object.seek(0)

conn.commit()
cur.close()
conn.close()

In [4]:
df = pd.read_csv(io_object, delimiter='|')

## understanding df

In [5]:
df.head()

,id,dt,client_catalog,item_catalog,sum,amount
0,17950747,2020-02-25 16:18:49,109732,10000,150.00,1.00
1,17950746,2020-02-25 16:18:49,109732,10042,270.00,1.00
2,17950745,2020-02-25 16:18:49,109732,9995,490.00,1.00
3,17950744,2020-02-25 16:18:49,109732,9666,205.00,1.00
4,17950743,2020-02-25 16:18:49,109732,9806,122.00,1.00


In [6]:
df.shape

(2281890, 6)

## preprocessing

```
generate rating
```

In [7]:
df1 = df.loc[df['amount'] > 0]
df2 = df.loc[df['amount'] < 0]

df1 = df1.groupby(['client_catalog', 'item_catalog'])[['item_catalog']].count().rename(columns={'item_catalog': 'number_of_purchases'}).reset_index()
df2 = df2.groupby(['client_catalog', 'item_catalog'])[['item_catalog']].count().rename(columns={'item_catalog': 'number_of_purchases'}).reset_index()

df = pd.merge(df1, df2, on=['client_catalog', 'item_catalog'], how='outer')
df = df.fillna(0)

df['rating'] = df['number_of_purchases_x'] - df['number_of_purchases_y']
df['rating'] = df['rating'].astype('int')

df.drop(columns=['number_of_purchases_x', 'number_of_purchases_y'], inplace=True)

```
reduce the dimensionality of the df set
```

In [8]:
# leave the products ordered (estimated) by more than 10 customers
min_item_orders = 10
filter_products = df['item_catalog'].value_counts() > min_item_orders
filter_products = filter_products[filter_products].index.tolist()

# leave customers who have uploaded (rated) more than 10 different products
min_clients_orders = 10
filter_clients = df['client_catalog'].value_counts() > min_clients_orders
filter_clients = filter_clients[filter_clients].index.tolist()

print('Size of the original dataframe: {}'.format(df.shape[0]))
number_of_clients = df['client_catalog'].nunique()
number_of_items = df['item_catalog'].nunique()
print('Number of clients: {}'.format(number_of_clients))
print('Number of products: {}'.format(number_of_items))
print()

clients_out = df[~df['client_catalog'].isin(filter_clients)]
clients_out = clients_out[['client_catalog']].drop_duplicates().reset_index(drop=True)

df = df[(df['item_catalog'].isin(filter_products)) & (df['client_catalog'].isin(filter_clients))].copy()

print('Size of the new dataframe: {}'.format(df.shape[0]))
number_of_clients = df['client_catalog'].nunique()
number_of_items = df['item_catalog'].nunique()
print('Number of clients: {}'.format(number_of_clients))
print('Number of products: {}'.format(number_of_items))

Size of the original dataframe: 686515
Number of clients: 10952
Number of products: 5618

Size of the new dataframe: 671465
Number of clients: 9155
Number of products: 4003


```
normalize ratings range
```

In [9]:
min_rating = min(df['rating'])

if min_rating < 0:
    df[['rating']] = df[['rating']] + abs(min_rating)

In [10]:
df.describe()

,client_catalog,item_catalog,rating
count,"671,465.00","671,465.00","671,465.00"
mean,"10,347.68","4,298.40",6.34
std,"12,189.28","9,678.78",4.51
min,2.00,3.00,0.00
25%,"3,122.00",932.00,4.00
50%,"8,946.00","1,889.00",5.00
75%,"14,231.00","5,988.00",7.00
max,"109,609.00","140,113.00",79.00


## work with the algorithm

```
surpise object
```

In [11]:
reader = Reader(rating_scale=(min(df['rating']), max(df['rating'])))
df_model = Dataset.load_from_df(df, reader)

```
compare algorithms
```

In [14]:
benchmark = []

for algorithm in [SVD(), KNNBaseline(), KNNBasic(), KNNWithMeans(), BaselineOnly()]:
    print('algo: ', algorithm)

    results = cross_validate(algorithm, df_model, measures=['RMSE'], cv=5, verbose=False)
    
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

algo:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x000001A868CF6438>
algo:  <surprise.prediction_algorithms.knns.KNNBaseline object at 0x000001A868CF6320>
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
algo:  <surprise.prediction_algorithms.knns.KNNBasic object at 0x000001A868CF6358>
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix

,test_rmse,fit_time,test_time
Algorithm,,,
SVD,3.27,30.86,1.33
BaselineOnly,3.53,2.57,1.46
KNNWithMeans,3.58,28.32,63.38
KNNBaseline,3.59,30.66,68.94
KNNBasic,4.01,26.84,58.88


```
tune SVD parameters with GridSearchCV
```

In [15]:
param_grid = {'n_epochs': [10, 20, 30], 'reg_all': [0.01, 0.02, 0.15]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=4)

gs.fit(df_model)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

results_cv = pd.DataFrame.from_dict(gs.cv_results).sort_values('mean_test_rmse')
results_cv[['mean_test_rmse', 'mean_test_mae', 'mean_fit_time', 'params']]

3.0550068032390563
{'n_epochs': 10, 'reg_all': 0.15}


,mean_test_rmse,mean_test_mae,mean_fit_time,params
2,3.06,1.87,14.95,"{'n_epochs': 10, 'reg_all': 0.15}"
5,3.08,1.87,29.82,"{'n_epochs': 20, 'reg_all': 0.15}"
8,3.09,1.88,43.96,"{'n_epochs': 30, 'reg_all': 0.15}"
1,3.16,1.94,15.19,"{'n_epochs': 10, 'reg_all': 0.02}"
0,3.19,1.96,15.55,"{'n_epochs': 10, 'reg_all': 0.01}"
4,3.28,2.03,29.32,"{'n_epochs': 20, 'reg_all': 0.02}"
7,3.32,2.07,45.09,"{'n_epochs': 30, 'reg_all': 0.02}"
3,3.33,2.07,29.82,"{'n_epochs': 20, 'reg_all': 0.01}"
6,3.39,2.12,43.44,"{'n_epochs': 30, 'reg_all': 0.01}"


```
CV for SVD with found hyperparameters
```

In [17]:
SVD_cv = SVD(n_epochs=10, reg_all=0.15)
cross_validate(SVD_cv, df_model, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.0618  3.0353  3.0435  3.0473  3.0361  3.0448  0.0096  
MAE (testset)     1.8606  1.8607  1.8635  1.8592  1.8618  1.8612  0.0015  
Fit time          15.29   16.90   15.94   16.72   15.40   16.05   0.66    
Test time         1.29    1.59    1.25    1.51    1.22    1.37    0.15    


{'test_rmse': array([3.06175146, 3.03529585, 3.04354088, 3.04729507, 3.03607116]),
 'test_mae': array([1.86058885, 1.8606822 , 1.86354428, 1.85916009, 1.86180159]),
 'fit_time': (15.28959345817566,
  16.89803433418274,
  15.944498300552368,
  16.72106122970581,
  15.400255918502808),
 'test_time': (1.2880022525787354,
  1.5930001735687256,
  1.2530426979064941,
  1.5070374011993408,
  1.2160451412200928)}

## model

In [12]:
trainset = df_model.build_full_trainset()

In [13]:
model_svd = SVD(n_epochs=10, reg_all=0.15)
model_svd.fit(trainset);

## get predictions

```
build antitestset
```

In [14]:
antitestset = pd.DataFrame(trainset.build_anti_testset(), columns=['client_catalog', 'item_catalog', 'other'])

antitestset['other'] = -1
antitestset = antitestset.astype('int32')

```
make chunks
```

In [15]:
antitestset_chunks = np.array_split(antitestset, 10)

```
make predictions in chunks
```

In [16]:
recommendations = pd.DataFrame()

for iteration, antitestset_chunk in enumerate(antitestset_chunks):
    print('antitestset_chunk number {} started'.format(iteration + 1))
    
    recommendations_chunk = pd.DataFrame(model_svd.test(antitestset_chunk.values))
    recommendations_chunk = recommendations_chunk[['uid', 'iid', 'est']]
    recommendations_chunk['est'] = round(recommendations_chunk['est'], 2)
    
    recommendations = pd.concat([recommendations, recommendations_chunk])

antitestset_chunk number 1 started
antitestset_chunk number 2 started
antitestset_chunk number 3 started
antitestset_chunk number 4 started
antitestset_chunk number 5 started
antitestset_chunk number 6 started
antitestset_chunk number 7 started
antitestset_chunk number 8 started
antitestset_chunk number 9 started
antitestset_chunk number 10 started


In [17]:
recommendations.describe()

,uid,iid,est
count,"35,976,000.00","35,976,000.00","35,976,000.00"
mean,"12,721.71","7,813.18",5.47
std,"15,966.34","17,307.52",1.29
min,2.00,3.00,0.00
25%,"4,167.00","1,526.00",4.70
50%,"10,302.00","4,053.00",5.18
75%,"15,144.00","8,026.00",5.85
max,"109,609.00","140,113.00",66.16


In [18]:
recommendations.shape

(35976000, 3)

In [19]:
recommendations.head(1)

,uid,iid,est
0,2,9,5.74


In [20]:
recommendations = recommendations.rename(columns={'uid': 'client_catalog', 'iid': 'item_catalog', 'est': 'rating'})

## get top 20 recommendations for each client

In [27]:
recommendations_top = recommendations.groupby('client_catalog').apply(lambda x: x.nlargest(20, 'rating')).reset_index(drop=True)

## get top items

In [28]:
items_top = recommendations_top['item_catalog'].value_counts().to_frame('count')
items_top = items_top.reset_index()
items_top = items_top.rename(columns={'index': 'item_catalog'})
items_top_20 = items_top[:20].copy()

## uninformative clients

In [29]:
items_top_20['key'] = 0
clients_out['key'] = 0

In [30]:
df_cartesian = clients_out.merge(items_top_20, how='outer')
df_cartesian = df_cartesian[['client_catalog', 'item_catalog']]
df_cartesian['rating'] = 'NaN'

In [31]:
recommendations_top['rating'] = recommendations_top['rating'].astype('object')

In [32]:
recommendations_top = pd.concat([recommendations_top, df_cartesian], sort=False)

In [33]:
recommendations_top

,client_catalog,item_catalog,rating
0,2,315,10.66
1,2,7893,10.54
2,2,859,9.37
3,2,156,9.33
4,2,8787,9.29
...,...,...,...
35675,16463,1236,NaN
35676,16463,1045,NaN
35677,16463,50,NaN
35678,16463,946,NaN


## push df to postgres

In [169]:
%%time
conn = psycopg2.connect(
    host="*****",
    user="*****",
    dbname="*****",
    password="*****",
    sslmode="require"
)
cur = conn.cursor()

product_catalog = pd.read_sql("SELECT * FROM product_catalog", conn)
client_catalog = pd.read_sql("SELECT * FROM client_catalog", conn)

Wall time: 5.98 s


In [170]:
recommendations_top = pd.merge(recommendations_top, product_catalog, on='item_catalog', how='left')
recommendations_top = pd.merge(recommendations_top, client_catalog, on='client_catalog', how='left')

items_top = pd.merge(items_top, product_catalog, on='item_catalog', how='left')

In [171]:
recommendations_top = recommendations_top[['client_id', 'item_id', 'rating']]
items_top = items_top[['item_id', 'count']]

In [172]:
recommendations_top

,client_id,item_id,rating
0,3b65b764-9fbd-11db-80d7-001438c58cb4,3192b1fa-509c-11e9-bba4-20677c60fed0,10.16
1,3b65b764-9fbd-11db-80d7-001438c58cb4,6db1b25d-9c05-11e9-bba6-20677c60fed0,9.25
2,3b65b764-9fbd-11db-80d7-001438c58cb4,dcc1e255-05c9-11e5-890b-40a8f02a150c,8.93
3,3b65b764-9fbd-11db-80d7-001438c58cb4,34df433c-b9c8-11e9-bba9-20677c60fed0,8.85
4,3b65b764-9fbd-11db-80d7-001438c58cb4,f19a5da7-46f0-11e9-bba3-20677c60fed0,8.66
...,...,...,...
219055,0f382055-2527-41d2-9a1a-11ff3ef04c4b,b60cba0b-2618-11ea-bbb0-20677c60fed0,NaN
219056,0f382055-2527-41d2-9a1a-11ff3ef04c4b,439b1756-2f7c-11e4-b5be-18a90562e070,NaN
219057,0f382055-2527-41d2-9a1a-11ff3ef04c4b,c44a0511-d39f-11e9-bbaa-20677c60fed0,NaN
219058,0f382055-2527-41d2-9a1a-11ff3ef04c4b,3b9221a3-00ee-11da-967e-505054503030,NaN


In [173]:
sio = StringIO()
recommendations_top.to_csv(sio, sep='\t', header=False, index=False)
sio.seek(0)

cur.execute("TRUNCATE recommendations")
cur.copy_from(sio, 'recommendations', columns=recommendations_top.columns)
conn.commit()

In [174]:
sio = StringIO()
items_top.to_csv(sio, sep='\t', header=False, index=False)
sio.seek(0)

cur.execute("TRUNCATE top")
cur.copy_from(sio, 'top', columns=items_top.columns)
conn.commit()

In [175]:
clients_out

,client_catalog,key
0,46,0
1,64,0
2,90,0
3,94,0
4,139,0
...,...,...
1766,11982,0
1767,16130,0
1768,16206,0
1769,16443,0
